<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/mistral_agent_traveltourism_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q colab-env
!pip install -q mistralai

In [2]:
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env # Assuming colab_env is for setting API key from environment

# Ensure colab-env and mistralai are installed
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai'.")
    exit()

# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic model for Itinerary Planning Agent's response format
class ItineraryResult(BaseModel):
    destination: str
    travel_dates: str
    daily_plan: list # List of daily activities/accommodations/transport
    estimated_cost: float
    recommendations: str

print("Creating AI agents for Tourism Travel Planning Domain...")

# --- Agent Definitions (Tourism Travel Planning Domain) ---

# 1. Destination Discovery Agent (Refactored from Compound Synthesis Agent)
destination_discovery_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for suggesting travel destinations based on user preferences and interests.",
    name="destination-discovery-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "suggest_destination",
                "description": "Suggest a travel destination based on criteria like budget, climate, activities, and travel companions.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "budget_range": {"type": "string", "description": "Budget range (e.g., 'low', 'medium', 'high')."},
                        "climate_preference": {"type": "string", "description": "Preferred climate (e.g., 'beach', 'mountains', 'city')."},
                        "activity_interests": {"type": "array", "items": {"type": "string"}, "description": "List of activity interests (e.g., 'hiking', 'museums', 'shopping')."}
                    },
                    "required": ["budget_range", "climate_preference", "activity_interests"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_destination_info",
                "description": "Retrieve general information about a specific travel destination, including popular attractions and best time to visit.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "destination_name": {"type": "string", "description": "Name of the destination (e.g., 'Paris', 'Kyoto')."}
                    },
                    "required": ["destination_name"]
                }
            }
        }
    ]
)
print(f"Destination Discovery Agent '{destination_discovery_agent.name}' created with ID: {destination_discovery_agent.id}")

# 2. Accommodation Booking Agent (Refactored from Target Binding Agent)
accommodation_booking_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for finding and booking accommodations (hotels, rentals) based on dates, location, and preferences.",
    name="accommodation-booking-agent",
    tools=[
        {"type": "web_search"}, # For searching external booking sites.
        {
            "type": "function",
            "function": {
                "name": "search_accommodations",
                "description": "Search for available hotels or rentals in a specific location for given dates and number of guests.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City or area for accommodation."},
                        "check_in_date": {"type": "string", "description": "Check-in date in YYYY-MM-DD format."},
                        "check_out_date": {"type": "string", "description": "Check-out date in YYYY-MM-DD format."},
                        "number_of_guests": {"type": "number", "description": "Number of guests."},
                        "type": {"type": "string", "enum": ["hotel", "apartment", "hostel", "all"], "description": "Type of accommodation (optional)."}
                    },
                    "required": ["location", "check_in_date", "check_out_date", "number_of_guests"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "book_accommodation",
                "description": "Book a specific accommodation based on its ID, guest details, and payment information.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "accommodation_id": {"type": "string", "description": "Unique ID of the accommodation to book."},
                        "guest_name": {"type": "string", "description": "Full name of the primary guest."},
                        "payment_info_token": {"type": "string", "description": "Mock payment token for booking."}
                    },
                    "required": ["accommodation_id", "guest_name", "payment_info_token"]
                }
            }
        }
    ]
)
print(f"Accommodation Booking Agent '{accommodation_booking_agent.name}' created with ID: {accommodation_booking_agent.id}")

# 3. Transportation Agent (Refactored from Drug Efficacy Prediction Agent)
transportation_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="transportation-agent",
    description="Agent for finding and booking flights, trains, or rental cars, and providing travel options.",
    instructions="Find best transportation options, compare prices, and suggest booking routes.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "transport_option_result",
                "schema": ItineraryResult.model_json_schema(), # Reusing ItineraryResult structure for transport options detail
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "search_flights",
                "description": "Search for available flights between origin and destination on specified dates.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin_airport": {"type": "string", "description": "Origin airport IATA code (e.g., 'YUL')."},
                        "destination_airport": {"type": "string", "description": "Destination airport IATA code (e.g., 'JFK')."},
                        "departure_date": {"type": "string", "description": "Departure date in YYYY-MM-DD format."},
                        "return_date": {"type": "string", "description": "Optional: Return date for round trip."}
                    },
                    "required": ["origin_airport", "destination_airport", "departure_date"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "search_rental_cars",
                "description": "Search for available rental cars at a location for given pick-up and drop-off dates.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "pickup_location": {"type": "string", "description": "City or airport code for car pickup."},
                        "pickup_date": {"type": "string", "description": "Pickup date in YYYY-MM-DD format."},
                        "dropoff_date": {"type": "string", "description": "Drop-off date in YYYY-MM-DD format."}
                    },
                    "required": ["pickup_location", "pickup_date", "dropoff_date"]
                }
            }
        }
    ]
)
print(f"Transportation Agent '{transportation_agent.name}' created with ID: {transportation_agent.id}")

# 4. Activity & Experience Agent (Refactored from Clinical Trial Planning Agent)
activity_experience_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for finding and recommending local activities, tours, and experiences based on destination and interests.",
    name="activity-experience-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "find_activities",
                "description": "Find popular activities or tours in a specific destination matching certain interests.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "destination_name": {"type": "string", "description": "Name of the destination."},
                        "interests": {"type": "array", "items": {"type": "string"}, "description": "List of interests (e.g., 'history', 'adventure', 'food')."}
                    },
                    "required": ["destination_name", "interests"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "book_tour_ticket",
                "description": "Book tickets for a specific tour or attraction for a given date and number of people.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "activity_id": {"type": "string", "description": "Unique ID of the activity/tour to book."},
                        "date": {"type": "string", "description": "Date for the activity in YYYY-MM-DD format."},
                        "number_of_people": {"type": "number", "description": "Number of people for the booking."}
                    },
                    "required": ["activity_id", "date", "number_of_people"]
                }
            }
        }
    ]
)
print(f"Activity & Experience Agent '{activity_experience_agent.name}' created with ID: {activity_experience_agent.id}")


# 5. Budgeting & Expense Agent (Refactored from Compound Library Agent)
budgeting_expense_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for estimating travel costs, tracking expenses, and providing budget recommendations.",
    name="budgeting-expense-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "estimate_trip_cost",
                "description": "Estimate the total cost for a trip based on destination, duration, and travel style.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "destination_name": {"type": "string", "description": "Name of the destination."},
                        "duration_days": {"type": "number", "description": "Duration of the trip in days."},
                        "travel_style": {"type": "string", "enum": ["budget", "mid-range", "luxury"], "description": "Preferred travel style."}
                    },
                    "required": ["destination_name", "duration_days", "travel_style"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_expense",
                "description": "Log a travel expense with its amount, category, and a brief description.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "amount": {"type": "number", "description": "Amount of the expense."},
                        "currency": {"type": "string", "description": "Currency of the expense (e.g., 'USD', 'EUR')."},
                        "category": {"type": "string", "description": "Category of the expense (e.g., 'food', 'transport', 'lodging')."},
                        "description": {"type": "string", "description": "Brief description of the expense."}
                    },
                    "required": ["amount", "currency", "category", "description"]
                }
            }
        }
    ]
)
print(f"Budgeting & Expense Agent '{budgeting_expense_agent.name}' created with ID: {budgeting_expense_agent.id}")


# 6. Itinerary Planning Agent (Refactored from Research Data Analysis Agent)
itinerary_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for creating detailed daily itineraries, suggesting routes between attractions, and optimizing schedules.",
    name="itinerary-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "create_daily_itinerary",
                "description": "Create a detailed daily itinerary for a trip, including attractions, meal times, and travel between points.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "destination_name": {"type": "string", "description": "Name of the destination."},
                        "travel_dates": {"type": "string", "description": "Travel dates (e.g., '2024-07-01 to 2024-07-05')."},
                        "interests": {"type": "array", "items": {"type": "string"}, "description": "List of traveler interests."}
                    },
                    "required": ["destination_name", "travel_dates", "interests"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "optimize_route",
                "description": "Optimize the travel route between a list of attractions or points of interest.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City or area where optimization is needed."},
                        "points_of_interest": {"type": "array", "items": {"type": "string"}, "description": "List of names of attractions or points of interest."}
                    },
                    "required": ["location", "points_of_interest"]
                }
            }
        }
    ]
)
print(f"Itinerary Planning Agent '{itinerary_planning_agent.name}' created with ID: {itinerary_planning_agent.id}")


# 7. Travel Document Agent (Refactored from Regulatory Compliance Agent)
travel_document_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for checking visa requirements, health advisories, and other essential travel documents for a destination.",
    name="travel-document-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_visa_requirements",
                "description": "Check visa requirements for a specific nationality traveling to a destination country.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "nationality": {"type": "string", "description": "Traveler's nationality (e.g., 'US', 'Canadian')."},
                        "destination_country": {"type": "string", "description": "Destination country (e.g., 'France', 'Japan')."}
                    },
                    "required": ["nationality", "destination_country"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_health_advisories",
                "description": "Retrieve health advisories and vaccination recommendations for a destination country.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "destination_country": {"type": "string", "description": "Destination country (e.g., 'Brazil', 'Thailand')."}
                    },
                    "required": ["destination_country"]
                }
            }
        }
    ]
)
print(f"Travel Document Agent '{travel_document_agent.name}' created with ID: {travel_document_agent.id}")


# 8. Customer Support Agent (Refactored from Mechanism of Action Analysis Agent)
customer_support_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for providing 24/7 travel assistance, answering FAQs, and rebooking services in case of disruptions.",
    name="customer-support-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "answer_faq",
                "description": "Provide answers to common travel-related frequently asked questions.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "The question to answer (e.g., 'What is checked baggage allowance?')."}
                    },
                    "required": ["question"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "rebook_travel_segment",
                "description": "Rebook a flight, hotel, or other travel segment due to disruption or change.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "booking_reference": {"type": "string", "description": "Original booking reference."},
                        "segment_type": {"type": "string", "enum": ["flight", "hotel", "car_rental"], "description": "Type of travel segment to rebook."},
                        "new_details": {"type": "string", "description": "New preferred details for rebooking (e.g., 'next available flight', 'hotel near airport')."}
                    },
                    "required": ["booking_reference", "segment_type", "new_details"]
                }
            }
        }
    ]
)
print(f"Customer Support Agent '{customer_support_agent.name}' created with ID: {customer_support_agent.id}")


# 9. Emergency Assistance Agent (Refactored from CRO/Vendor Coordination Agent)
emergency_assistance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for providing immediate assistance in travel emergencies, including lost passports, medical issues, or natural disasters.",
    name="emergency-assistance-agent",
    tools=[
        {"type": "web_search"}, # For finding local embassy contacts or emergency services.
        {
            "type": "function",
            "function": {
                "name": "locate_nearest_embassy",
                "description": "Locate the nearest embassy or consulate for a given nationality in a specific city/country.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "nationality": {"type": "string", "description": "Traveler's nationality."},
                        "current_location": {"type": "string", "description": "Current city or country of the traveler."}
                    },
                    "required": ["nationality", "current_location"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "contact_emergency_services",
                "description": "Initiate contact with local emergency services (e.g., police, ambulance) in case of a critical situation.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "emergency_type": {"type": "string", "description": "Type of emergency (e.g., 'medical', 'security')."},
                        "location_details": {"type": "string", "description": "Specific location details for emergency responders."}
                    },
                    "required": ["emergency_type", "location_details"]
                }
            }
        }
    ]
)
print(f"Emergency Assistance Agent '{emergency_assistance_agent.name}' created with ID: {emergency_assistance_agent.id}")


print("\nAll new tourism travel planning-related agents have been defined.")

# --- Mock functions for tourism travel planning domain tools ---
# These mock functions provide simulated responses for the new agents' tools.

def suggest_destination(budget_range: str, climate_preference: str, activity_interests: list):
    """MOCK function to suggest a travel destination."""
    print(f"\n[DEBUG] MOCK CALL: suggest_destination - Budget: {budget_range}, Climate: {climate_preference}, Activities: {activity_interests}")
    if "beach" in climate_preference.lower() and "low" in budget_range.lower():
        return {"status": "success", "destination": "Bali, Indonesia", "message": "Relaxing beach destination on a budget."}
    if "mountains" in climate_preference.lower() and "hiking" in [a.lower() for a in activity_interests]:
        return {"status": "success", "destination": "Swiss Alps, Switzerland", "message": "Stunning mountains for hiking."}
    return {"status": "success", "destination": "Tokyo, Japan", "message": "Vibrant city with diverse activities."}

def get_destination_info(destination_name: str):
    """MOCK function to get destination information."""
    print(f"\n[DEBUG] MOCK CALL: get_destination_info for: {destination_name}")
    if "paris" in destination_name.lower():
        return {"status": "success", "info": "Paris is known for the Eiffel Tower, Louvre Museum, and exquisite cuisine. Best time to visit: Spring or Autumn.", "attractions": ["Eiffel Tower", "Louvre Museum", "Notre Dame"]}
    return {"status": "not_found", "message": "Destination info not found."}

def search_accommodations(location: str, check_in_date: str, check_out_date: str, number_of_guests: int, type: str = "all"):
    """MOCK function to search for accommodations."""
    print(f"\n[DEBUG] MOCK CALL: search_accommodations - Location: {location}, Dates: {check_in_date} to {check_out_date}, Guests: {number_of_guests}, Type: {type}")
    if "london" in location.lower() and number_of_guests == 2:
        return {"status": "success", "accommodations": [
            {"id": "HOTEL-001", "name": "Central London Hotel", "price_per_night": 200, "type": "hotel"},
            {"id": "APT-002", "name": "Cozy London Apartment", "price_per_night": 150, "type": "apartment"}
        ], "message": "Found available accommodations."}
    return {"status": "no_results", "message": "No accommodations found matching criteria."}

def book_accommodation(accommodation_id: str, guest_name: str, payment_info_token: str):
    """MOCK function to book accommodation."""
    print(f"\n[DEBUG] MOCK CALL: book_accommodation - ID: {accommodation_id}, Guest: {guest_name}, Payment Token: {payment_info_token}")
    return {"status": "booked", "booking_id": "BKG-001", "accommodation_id": accommodation_id, "guest": guest_name, "message": "Accommodation successfully booked."}

def search_flights(origin_airport: str, destination_airport: str, departure_date: str, return_date: str = None):
    """MOCK function to search for flights."""
    print(f"\n[DEBUG] MOCK CALL: search_flights - From: {origin_airport}, To: {destination_airport}, Dept: {departure_date}, Ret: {return_date}")
    if origin_airport.upper() == "YUL" and destination_airport.upper() == "CDG":
        return {"status": "success", "flights": [
            {"flight_number": "AF123", "airline": "Air France", "price": 750, "departure_time": "14:00"},
            {"flight_number": "AC456", "airline": "Air Canada", "price": 780, "departure_time": "15:30"}
        ], "message": "Flights found."}
    return {"status": "no_flights", "message": "No flights found."}

def search_rental_cars(pickup_location: str, pickup_date: str, dropoff_date: str):
    """MOCK function to search for rental cars."""
    print(f"\n[DEBUG] MOCK CALL: search_rental_cars - Location: {pickup_location}, Pickup: {pickup_date}, Dropoff: {dropoff_date}")
    if "rome" in pickup_location.lower():
        return {"status": "success", "cars": [{"rental_company": "Hertz", "car_type": "Compact", "price_per_day": 40}], "message": "Rental cars found."}
    return {"status": "no_cars", "message": "No rental cars found."}

def find_activities(destination_name: str, interests: list):
    """MOCK function to find activities."""
    print(f"\n[DEBUG] MOCK CALL: find_activities - Destination: {destination_name}, Interests: {interests}")
    if "paris" in destination_name.lower() and "museums" in [i.lower() for i in interests]:
        return {"status": "success", "activities": [{"id": "ACT-001", "name": "Louvre Museum Tour", "price": 50}], "message": "Activities found."}
    return {"status": "no_activities", "message": "No activities found."}

def book_tour_ticket(activity_id: str, date: str, number_of_people: int):
    """MOCK function to book tour tickets."""
    print(f"\n[DEBUG] MOCK CALL: book_tour_ticket - Activity: {activity_id}, Date: {date}, People: {number_of_people}")
    return {"status": "booked", "ticket_id": "TKT-001", "activity": activity_id, "message": "Tickets booked."}

def estimate_trip_cost(destination_name: str, duration_days: int, travel_style: str):
    """MOCK function to estimate trip cost."""
    print(f"\n[DEBUG] MOCK CALL: estimate_trip_cost - Dest: {destination_name}, Duration: {duration_days}, Style: {travel_style}")
    base_cost_per_day = 100 # budget
    if travel_style == "mid-range":
        base_cost_per_day = 250
    elif travel_style == "luxury":
        base_cost_per_day = 500
    total_cost = base_cost_per_day * duration_days
    return {"status": "success", "estimated_cost": total_cost, "currency": "USD", "message": f"Estimated cost for {destination_name}."}

def log_expense(amount: float, currency: str, category: str, description: str):
    """MOCK function to log travel expense."""
    print(f"\n[DEBUG] MOCK CALL: log_expense - Amount: {amount} {currency}, Category: {category}, Desc: {description}")
    return {"status": "logged", "expense_id": "EXP-001", "message": "Expense logged."}

def create_daily_itinerary(destination_name: str, travel_dates: str, interests: list):
    """MOCK function to create daily itinerary."""
    print(f"\n[DEBUG] MOCK CALL: create_daily_itinerary - Dest: {destination_name}, Dates: {travel_dates}, Interests: {interests}")
    if "paris" in destination_name.lower():
        return {
            "status": "success",
            "itinerary": {
                "day_1": "Morning: Eiffel Tower, Afternoon: Louvre Museum, Evening: Seine River cruise.",
                "day_2": "Morning: Notre Dame, Afternoon: Montmartre and Sacré-Cœur, Evening: Dinner in Latin Quarter."
            },
            "message": "Itinerary created for Paris."
        }
    return {"status": "no_itinerary", "message": "Could not create detailed itinerary."}

def optimize_route(location: str, points_of_interest: list):
    """MOCK function to optimize travel route."""
    print(f"\n[DEBUG] MOCK CALL: optimize_route - Location: {location}, POIs: {points_of_interest}")
    return {"status": "success", "optimized_route": ["POI 1", "POI 3", "POI 2"], "message": "Route optimized."}

def check_visa_requirements(nationality: str, destination_country: str):
    """MOCK function to check visa requirements."""
    print(f"\n[DEBUG] MOCK CALL: check_visa_requirements - Nationality: {nationality}, Dest Country: {destination_country}")
    if nationality.lower() == "us" and destination_country.lower() == "france":
        return {"status": "no_visa_required", "details": "US citizens do not require a visa for short stays in France.", "message": "Visa info retrieved."}
    return {"status": "visa_required", "details": "Visa required. Check embassy website.", "message": "Visa info retrieved."}

def get_health_advisories(destination_country: str):
    """MOCK function to get health advisories."""
    print(f"\n[DEBUG] MOCK CALL: get_health_advisories - Country: {destination_country}")
    if "brazil" in destination_country.lower():
        return {"status": "advisory_issued", "advisories": ["Yellow fever vaccination recommended.", "Zika virus alert."], "message": "Health advisories found."}
    return {"status": "no_advisories", "message": "No specific health advisories found."}

def answer_faq(question: str):
    """MOCK function to answer FAQs."""
    print(f"\n[DEBUG] MOCK CALL: answer_faq - Question: '{question}'")
    if "baggage allowance" in question.lower():
        return {"status": "success", "answer": "Checked baggage allowance varies by airline and ticket class. Please check your airline's policy.", "faq_topic": "Baggage"}
    return {"status": "not_found", "answer": "Sorry, I don't have an answer to that specific FAQ.", "faq_topic": "N/A"}

def rebook_travel_segment(booking_reference: str, segment_type: str, new_details: str):
    """MOCK function to rebook travel segment."""
    print(f"\n[DEBUG] MOCK CALL: rebook_travel_segment - Booking: {booking_reference}, Type: {segment_type}, Details: '{new_details}'")
    return {"status": "rebooked", "new_booking_id": "RBK-001", "message": "Travel segment successfully rebooked."}

def locate_nearest_embassy(nationality: str, current_location: str):
    """MOCK function to locate nearest embassy."""
    print(f"\n[DEBUG] MOCK CALL: locate_nearest_embassy - Nationality: {nationality}, Location: {current_location}")
    if nationality.lower() == "us" and "paris" in current_location.lower():
        return {"status": "success", "embassy_info": "US Embassy in Paris, France. Address: 2 Avenue Gabriel, 75008 Paris. Phone: +33-1-43-12-22-22.", "message": "Embassy found."}
    return {"status": "not_found", "message": "Embassy not found for specified nationality/location."}

def contact_emergency_services(emergency_type: str, location_details: str):
    """MOCK function to contact emergency services."""
    print(f"\n[DEBUG] MOCK CALL: contact_emergency_services - Type: {emergency_type}, Location: '{location_details}'")
    return {"status": "alert_sent", "service_contacted": "Local Police/Ambulance", "message": "Emergency services notified."}

def expedite_order(order_id: str, reason_for_urgency: str): # Reusing expedite_order for emergency context if needed
    """MOCK function to expedite a request (e.g., replacement passport)."""
    print(f"\n[DEBUG] MOCK CALL: expedite_request - Item/Order: {order_id}, Reason: '{reason_for_urgency}'")
    return {"status": "expedite_requested", "request_id": order_id, "message": "Expedite request initiated."}


# --- Master Tool Executor Mapping (Updated for Tourism Travel Planning Domain) ---
tool_executor = {
    "suggest_destination": suggest_destination,
    "get_destination_info": get_destination_info,
    "search_accommodations": search_accommodations,
    "book_accommodation": book_accommodation,
    "search_flights": search_flights,
    "search_rental_cars": search_rental_cars,
    "find_activities": find_activities,
    "book_tour_ticket": book_tour_ticket,
    "estimate_trip_cost": estimate_trip_cost,
    "log_expense": log_expense,
    "create_daily_itinerary": create_daily_itinerary,
    "optimize_route": optimize_route,
    "check_visa_requirements": check_visa_requirements,
    "get_health_advisories": get_health_advisories,
    "answer_faq": answer_faq,
    "rebook_travel_segment": rebook_travel_segment,
    "locate_nearest_embassy": locate_nearest_embassy,
    "contact_emergency_services": contact_emergency_services,
    "expedite_order": expedite_order, # Can be repurposed for urgent requests in emergency
    "internal_web_search_tool": lambda *args, **kwargs: "Mock web search: General travel information retrieved."
}

# --- Function to standardize tools for client.chat.complete (remains the same) ---
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# --- Test Case Execution for Tourism Travel Planning Agents ---

print("\n--- Executing Test Cases for Tourism Travel Planning Agents (via chat completions) ---")

test_cases = [
    {
        "agent": destination_discovery_agent,
        "name": "Destination Discovery Agent",
        "query": "Suggest a destination for a medium budget, beach preference, and interest in relaxation.",
        "expected_tool_call": "suggest_destination"
    },
    {
        "agent": destination_discovery_agent,
        "name": "Destination Discovery Agent",
        "query": "Tell me about Paris.",
        "expected_tool_call": "get_destination_info"
    },
    {
        "agent": accommodation_booking_agent,
        "name": "Accommodation Booking Agent",
        "query": "Search for hotels in London for 2 guests, checking in 2025-08-01 and checking out 2025-08-05.",
        "expected_tool_call": "search_accommodations"
    },
    {
        "agent": accommodation_booking_agent,
        "name": "Accommodation Booking Agent",
        "query": "Book HOTEL-001 for John Doe, payment token 'XYZ123'.",
        "expected_tool_call": "book_accommodation"
    },
    {
        "agent": transportation_agent,
        "name": "Transportation Agent",
        "query": "Find flights from Montreal (YUL) to Paris (CDG) departing 2025-09-10.",
        "expected_tool_call": "search_flights"
    },
    {
        "agent": transportation_agent,
        "name": "Transportation Agent",
        "query": "Search for rental cars in Rome from 2025-10-01 to 2025-10-07.",
        "expected_tool_call": "search_rental_cars"
    },
    {
        "agent": activity_experience_agent,
        "name": "Activity & Experience Agent",
        "query": "Find museum activities in Paris.",
        "expected_tool_call": "find_activities"
    },
    {
        "agent": activity_experience_agent,
        "name": "Activity & Experience Agent",
        "query": "Book 2 tickets for ACT-001 on 2025-09-15.",
        "expected_tool_call": "book_tour_ticket"
    },
    {
        "agent": budgeting_expense_agent,
        "name": "Budgeting & Expense Agent",
        "query": "Estimate cost for a 7-day mid-range trip to Rome.",
        "expected_tool_call": "estimate_trip_cost"
    },
    {
        "agent": budgeting_expense_agent,
        "name": "Budgeting & Expense Agent",
        "query": "Log expense: $50 USD for food, dinner in local restaurant.",
        "expected_tool_call": "log_expense"
    },
    {
        "agent": itinerary_planning_agent,
        "name": "Itinerary Planning Agent",
        "query": "Create a daily itinerary for a trip to Paris from 2025-09-10 to 2025-09-14, interested in history and art.",
        "expected_tool_call": "create_daily_itinerary"
    },
    {
        "agent": itinerary_planning_agent,
        "name": "Itinerary Planning Agent",
        "query": "Optimize route for Eiffel Tower, Louvre Museum, Notre Dame in Paris.",
        "expected_tool_call": "optimize_route"
    },
    {
        "agent": travel_document_agent,
        "name": "Travel Document Agent",
        "query": "Check visa requirements for US citizens traveling to France.",
        "expected_tool_call": "check_visa_requirements"
    },
    {
        "agent": travel_document_agent,
        "name": "Travel Document Agent",
        "query": "Get health advisories for Brazil.",
        "expected_tool_call": "get_health_advisories"
    },
    {
        "agent": customer_support_agent,
        "name": "Customer Support Agent",
        "query": "What is the checked baggage allowance?",
        "expected_tool_call": "answer_faq"
    },
    {
        "agent": customer_support_agent,
        "name": "Customer Support Agent",
        "query": "Rebook flight for booking ABCDEF to the next available flight.",
        "expected_tool_call": "rebook_travel_segment"
    },
    {
        "agent": emergency_assistance_agent,
        "name": "Emergency Assistance Agent",
        "query": "Locate nearest US embassy in Paris.",
        "expected_tool_call": "locate_nearest_embassy"
    },
    {
        "agent": emergency_assistance_agent,
        "name": "Emergency Assistance Agent",
        "query": "Contact emergency services for a medical emergency at 123 Main St, Paris.",
        "expected_tool_call": "contact_emergency_services"
    },
    {
        "agent": destination_discovery_agent, # Reusing for web search example
        "name": "Destination Discovery Agent (Web Search Example)",
        "query": "Popular tourist attractions in Rome.",
        "expected_tool_call": "internal_web_search_tool"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"] # This is for tracking, not direct control.

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")

    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        # Get the API-compatible tool list for the current agent.
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        print(f" [DEBUG] Sending initial user query to the {agent_name}...")
        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f" Tool Name: {tool_call.function.name}")
                print(f" Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None
                # Check if the proposed tool exists in our local executor mapping.
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        # Execute the local mock function based on its name.
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output_content}")
                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                # Add the tool output message to the conversation history.
                # This is crucial for the model to "see" the result of the tool call.
                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id # Links the tool output to the specific call request.
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            # Second turn: Send the conversation history (including tool outputs) back to the model.
            # The model will then generate a final response based on the tool's output.
            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list, # Tools must be provided in all calls if they are part of the conversation context.
            )

            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)
            # Add the final assistant response to history (optional for a single-turn demo, but good practice).
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)

        else:
            # If no tool calls were proposed in the first turn, print the direct response.
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")
        print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime) might help.")

print("\n--- All test cases execution complete. ---")


Mounted at /content/gdrive
Creating AI agents for Tourism Travel Planning Domain...
Destination Discovery Agent 'destination-discovery-agent' created with ID: ag_06856c0f62567b068000ac036f22f235
Accommodation Booking Agent 'accommodation-booking-agent' created with ID: ag_06856c0f699c7dc1800044fdd922ab4e
Transportation Agent 'transportation-agent' created with ID: ag_06856c0f6d0b79a48000861e3bcc231d
Activity & Experience Agent 'activity-experience-agent' created with ID: ag_06856c0f702077b9800062c4707dcd53
Budgeting & Expense Agent 'budgeting-expense-agent' created with ID: ag_06856c0f733879bf8000472a7646c8e5
Itinerary Planning Agent 'itinerary-planning-agent' created with ID: ag_06856c0f7640732180003b296f20cfe3
Travel Document Agent 'travel-document-agent' created with ID: ag_06856c0f79d7774b80002a8475fbc934
Customer Support Agent 'customer-support-agent' created with ID: ag_06856c0f7cd4755f80008d36f782ea66
Emergency Assistance Agent 'emergency-assistance-agent' created with ID: ag_06